In [55]:
import pandas as pd
import json
import random

In [56]:
# Replace 'data.jsonl' with the path to your JSONL file
file_path = 'data/valid.jsonl'

# Read the JSON Lines file into a list of dictionaries
data = []
with open(file_path, 'r') as file:
    for line in file:
        data.append(json.loads(line))

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(data)
def extract_first_part(text):
    parts = text.split('.')
    if len(parts) > 0:
        return parts[0]
    else:
        return ''



In [57]:
file_path = 'data/valid-labels.lst'  # Replace this with the actual path to your file

# Read the file
with open(file_path, 'r') as file:
    labels = file.readlines()

# Display the labels
label_list = []
for label in labels:
   label_list.append(label.strip())  # .strip() removes any leading/trailing whitespaces or newlines
df['label'] = label_list
# Apply the function to create a new column
df['first_part'] = df['ctx_a'].apply(extract_first_part)
sampled_df = df.sample(n=1000, random_state=42)
sampled_df = sampled_df.reset_index(drop=True)

In [58]:
sampled_df

,ind,id,activity_label,ctx_a,ctx_b,ctx,dataset,split_type,ending_options,label,first_part
0,19073,31450,Personal Care and Style,[header] How to get perfect eyebrows [title] F...,,[header] How to get perfect eyebrows [title] F...,wikihow,zeroshot,[Measure the length of your inner brow and the...,3,[header] How to get perfect eyebrows [title] F...
1,20017,33062,Personal Care and Style,[header] How to make french knot easy way hair...,,[header] How to make french knot easy way hair...,wikihow,zeroshot,"[For this reason, french knot style hair will ...",3,[header] How to make french knot easy way hair...
2,34560,v_pHAIrx_cRFU,Sharpening knives,The person puts a new belt on the machine. The...,the person,The person puts a new belt on the machine. The...,activitynet,zeroshot,"[puts the sword back on the machine., removes ...",3,The person puts a new belt on the machine
3,41427,68173,Family Life,[header] How to plan your own funeral [title] ...,,[header] How to plan your own funeral [title] ...,wikihow,zeroshot,[[title] Prepare for your funeral by creating ...,3,[header] How to plan your own funeral [title] ...
4,9755,16080,Health,[header] How to buy an exercise bike [title] E...,,[header] How to buy an exercise bike [title] E...,wikihow,indomain,[You will want to check with your trainer befo...,3,[header] How to buy an exercise bike [title] E...
...,...,...,...,...,...,...,...,...,...,...,...
995,42169,69383,Family Life,[header] How to inspire creativity in your kid...,,[header] How to inspire creativity in your kid...,wikihow,zeroshot,[[substeps] Consider starting your own art cla...,3,[header] How to inspire creativity in your kid...
996,18722,30839,Home and Garden,[header] How to be a domestic goddess [title] ...,,[header] How to be a domestic goddess [title] ...,wikihow,indomain,[[substeps] Organized and set your hours so th...,2,[header] How to be a domestic goddess [title] ...
997,16318,v_T_q3f10pkOg,Playing water polo,Two teams play water volleyball in an outdoor ...,the two teams,Two teams play water volleyball in an outdoor ...,activitynet,indomain,[play a blue and yellow game in the indoor poo...,2,Two teams play water volleyball in an outdoor ...
998,11517,18997,Family Life,[header] How to choose toys that get kids movi...,,[header] How to choose toys that get kids movi...,wikihow,zeroshot,[[substeps] Keep in mind that kids can have ma...,1,[header] How to choose toys that get kids movi...


In [59]:
slot_ans_list = []
slot_question_list = []
for index,row in sampled_df.iterrows():
    sentence = row['ctx']
    ans_list = row['ending_options']
    ans_str = ""
    print(ans_list)
    for i in range(len(ans_list)):
        ans_str += "option"+ str(i)+ ": " + ans_list[i]
        if i != len(ans_list)-1:
            ans_str += "\n"
    print(ans_str)
    # ans_str = "A:"+ans_list[0] + "\n"+ "B:"+ans_list[1] + "\n"+ "C:"+ans_list[2] + "\n"+ "D:"+ans_list[3]
    slot_answer = ans_list[int(row['label'])]
    ans_list.remove(slot_answer)
    wrong_ans = random.choice(ans_list)
    slot_question =  (sentence + "\n" + ans_str).replace(wrong_ans,"[]")
    slot_ans_list.append(wrong_ans)
    slot_question_list.append(slot_question)
    print(slot_question,wrong_ans)
    

['Measure the length of your inner brow and the width of your outer brow. Make sure it is inside the outer portion of your eye.', 'It should create a slight curve out the corner. Lower it back into position and not over your eyes.', 'Draw a dot just above or just above the inner corner of your eye. Make sure that the lines cross a little bit above or below your eyebrows.', 'The place where the instrument overlaps your brow is where it should begin. Mark the spot by making a dot there with an eyeliner pencil.']
option0: Measure the length of your inner brow and the width of your outer brow. Make sure it is inside the outer portion of your eye.
option1: It should create a slight curve out the corner. Lower it back into position and not over your eyes.
option2: Draw a dot just above or just above the inner corner of your eye. Make sure that the lines cross a little bit above or below your eyebrows.
option3: The place where the instrument overlaps your brow is where it should begin. Mark t

In [62]:
sampled_df['slot_question'] = slot_question_list
sampled_df['slot_answer'] = slot_ans_list
sampled_df.to_csv("data/mc.csv")